# Convolutional Neural Network in Pytorch
- following this guide: `https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/`

In [2]:
from torch.nn import (
    Module, 
    Conv2d, 
    Linear, 
    MaxPool2d, 
    ReLU, 
    LogSoftmax
)
from torch import flatten

/home/mmazeem/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class LeNet(Module):
    def __init__(self, num_channels, classes):
        super().__init__()

        # First set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(
            in_channels=num_channels,   # Channels in input image
            out_channels=20,            # Number of filters
            kernel_size=(5,5)           # Size of each filter
        )
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2,2), stride=(2,2))

        # Second set of CONV => RELU => POOL lyaers
        self.conv2 = Conv2d(
            in_channels=20,             # Output channels from first conv
            out_channels=50,            # Number of filters
            kernel_size=(5,5)           # Size of each filter
        )
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2,2), stride=(2,2))

        # FC => RELU layers
        self.fc1 = Linear(
            in_features=800,            # num of input = flattened conv output
            out_features=500            # num of hidden neurons
        )
        self.relu3 = ReLU()

        # Softmax classifier
        self.fc2 = Linear(
            in_features=500,            # num of hidden neurons
            out_features=classes        # num of output neurons
        )
        self.log_softmax = LogSoftmax(dim=1)


        def forward(self, x):
            # First set of conv layers
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.maxpool1(x)

            # Second set of conv layers
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.maxpool2(x)

            # Flatten output and pass it to dense layers
            x = flatten(x, 1)
            x = self.fc1(x)
            x = self.relu3(x)

            # Softmax classifier
            x = self.fc2(x)
            output = self.log_softmax(x)

            return output

## Train

In [4]:
import matplotlib
matplotlib.use("Agg")

from sklearn.metrics import classification_report
from torch.utils.data import (random_split, DataLoader)
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch
import time

In [5]:
# Training hyperparameters
init_lr = 1e-3
batch_size = 64
epochs = 10

# train and val split
train_split = 0.75
val_split = 1 - train_split

# Get device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Prepare Dataset

In [ ]:
train_data = KMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = KMNIST(root="data", train=False, download=True, transform=ToTensor())

num_train_samples = int(len(train_data) * train_split)
num_val_samples = int(len(train_data) * val_split)
(train_data, val_data) = random_split(
    train_data,
	[num_train_samples, num_val_samples],
	generator=torch.Generator().manual_seed(42)
)

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

train_steps = len(train_dataloader.dataset) // batch_size
val_steps = len(val_dataloader.dataset) // batch_size
